In [ ]:
#generating tifs for heatmap timelapse

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Create a directory to save the TIFF images if it doesn't exist
output_dir = "heatmap_tifs"
os.makedirs(output_dir, exist_ok=True)

# Frame numbers to process
frame_numbers = range(first, last)

# Loop through each frame number and generate the heatmap
for frame_number in frame_numbers:
    # Construct the file names
    vx_file = f"Velocity_Map_X_Frame_{frame_number}.xlsx"
    vy_file = f"Velocity_Map_Y_Frame_{frame_number}.xlsx"
    
    # Read data from the Excel files
    df_vx = pd.read_excel(vx_file, header=None)
    df_vy = pd.read_excel(vy_file, header=None)
    
    # Convert the velocity components and compute velocity magnitude
    vxt = df_vx.values * 5.86
    vyt = df_vy.values * 5.86
    vmag = np.sqrt(vxt**2 + vyt**2)
    
    # Define coordinates for plotting
    x, y = np.meshgrid(np.arange(vxt.shape[1]), np.arange(vxt.shape[0]))
    
    # Create a figure and axis for the heatmap
    fig, ax1 = plt.subplots(figsize=(10, 8))
    
    # Create the heatmap
    im = ax1.imshow(vmag, cmap='turbo', extent=[0, 150, 110, 0], vmax=max)
    
    # Set the axis limits to match the data extent
    ax1.set_ylim(ax1.get_ylim()[::-1])  # Invert the y-axis
    
    # Add a colorbar
    cb = fig.colorbar(im, ax=ax1, pad=0.02)
    
    # Set labels and title
    ax1.set_xlabel('X Coordinate')
    ax1.set_ylabel('Y Coordinate')
    ax1.set_title(f'Velocity Magnitude Heatmap for Frame {frame_number}')
    
    # Save the plot as a TIFF file
    tiff_path = os.path.join(output_dir, f"heatmap_frame_{frame_number:04d}.tif")
    plt.savefig(tiff_path, bbox_inches='tight', dpi=300, format='tiff')  # Save as TIFF with high resolution
    plt.close(fig)  # Close the figure to free up memory

print(f"Heatmap TIFFs saved in the '{output_dir}' directory.")

In [ ]:
#converting heatmap tifs to avi

import cv2
import os
import glob

# Directory containing the TIFF files
input_dir = "heatmap_tifs"

# Output AVI file name
output_avi = "heatmap_timelapse_mjpg.avi"

# Frame rate for the video
frame_rate = 5  # frames per second (adjust as needed)

# Get a list of all TIFF files in the directory, sorted by filename
tiff_files = sorted(glob.glob(os.path.join(input_dir, "*.tif")))

# Check if there are TIFF files to process
if not tiff_files:
    print("No TIFF files found in the directory.")
    exit()

# Read the first TIFF file to get the frame size
first_frame = cv2.imread(tiff_files[0])
height, width, layers = first_frame.shape

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'MJPG')  # Using MJPG codec
video_writer = cv2.VideoWriter(output_avi, fourcc, frame_rate, (width, height))

# Process each TIFF file and write to the video
for tiff_file in tiff_files:
    frame = cv2.imread(tiff_file)  # Read the TIFF file
    video_writer.write(frame)  # Write the frame to the video

# Release the VideoWriter object
video_writer.release()

print(f"Video saved as {output_avi}")